### Based on the following:

1. In the past twelve months, did you receive free food or meals?
2. In the past twelve months, were you ever hungry, but didn't eat because you couldn't afford enough food?
3. In the past twelve months, did you ever not pay the full amount of rent or mortgage payments?
4. In the past twelve months, were you evictd from your home or apartment for not paying the rent or mortgage?
5. In the past twelve months, did you not pay the full amount of gas, oil, or electricity bill?
6. In the past twelve months, was your gas or electric services ever turned off, or the heating oil company did not deliver oil, because there wasn't enough money to pay the bills?
7. In the past twelve months, did you move in with other people even for a little while because of financial problems?
8. In the past twelve months, did you stay at a shelter, in an abandoned building, an automobile or any other place not menat for regular housing, even for one night?
9. IN the past twelve months, was there anyone in your household who needed to see a doctor or go to the hospital but couldn't go because of cost?
10. IN the past twelve months, was your telephone service cancelled or disconnected by the telephone company because there wasn't enough money to pay the bill?

In [ ]:
import pandas as pd
from sklearn import feature_selection, tree
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC, SVC
from sklearn.linear_model import BayesianRidge, LinearRegression, LassoCV
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
import numpy as np

from itertools import product, combinations
from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
# Get Data for Training...

data_dir = '../.data'
fp_train = '{}/train.csv'.format(data_dir)
df_test = pd.read_csv(fp_train,index_col=0)

In [ ]:
# Get Feature Data...

df_path = "./../FFChallenge_v2/background.dta"

df_features = None
with open(df_path, "r") as f:
    df_features = pd.read_stata(f)
    print df_features.head()
    
df_train = df_features.set_index('challengeID')
df_train_na = df_train.replace('NA', np.NaN)
df_train_na.cf4fint = pd.to_datetime(df_train_na.cf4fint)
df_train_na_cols = df_train_na.columns[df_train_na.dtypes == 'object']
df_train_na[df_train_na_cols] = df_train_na[df_train_na_cols].apply(lambda x: pd.to_numeric(x, errors = 'ignore'))

df_train_no_obj = df_train_na[df_train_na.columns[df_train_na.dtypes != 'object']]
final_cols = df_train_no_obj.columns[~ df_train_no_obj.isnull().all()]
df_final = pd.DataFrame(df_train_no_obj[final_cols])
print df_final.shape

# Find number of unique values in each column. If unique == 1, then remove from final data frame.
n = df_final.apply(lambda x: len(x.unique()))
df_final = pd.DataFrame(df_final[df_final.columns[n>1]])
print df_final.shape

In [ ]:
cols_interest = [
#    'm2h19e',
#    'm2h19d',
#    'm2h19f',
#    'm2h19g',
#    'm2h19h',
#    'f2h17e',
 #   'f2h17d',
#    'f2h17f',
#    'f2h17g',
#    'f2h17h',
    'm2h19a',
    'm2h19b',
    'm2h19c',
    'm2h19d',
    'm2h19e',
    'm2h19f',
    'm2h19g',
    'm2h19h',
    'm2h19i',
    'm2h19j',
    'm2h19k',
#    'f3i23c',
#    'f3i23b',
#    'f3i23d',
    'm4i23a',
    'm4i23b',
    'm4i23c',
    'm4i23d',
    'm4i23e',
    'm4i23f',
    'm4i23g',
    'm4i23h',
    'm4i23i',
    'm4i23j',
    'm4i23k',
    'm5f23a',
    'm5f23b',
    'm5f23c',
    'm5f23d',
    'm5f23e',
    'm5f23f',
    'm5f23g',
    'm5f23h',
    'm5f23i',
    'm5f23j',
    'm5f23k',
#    'n5g1a',
#    'n5g1b',
#    'n5g1c',
#    'n5g1d',
#    'n5g1e',
#    'n5g1f',
#    'n5g1g',
#    'n5g1h',
#    'n5g1i',
#    'n5g1j',
]

liveIn = pd.DataFrame(df_final.m5a2.apply(lambda x: int(not x.startswith('1'))))

df = pd.DataFrame(df_final[cols_interest])
df = df.applymap(lambda x: int(x.split(' ')[0]))
df = df.applymap(lambda x: x * (x >= 0))

df['cum_sum'] = df.sum(axis=1)
df['mat_hds'] = df[cols_interest[22:]].apply(lambda x: x[x==1].sum()/11., axis=1)
df['mat_hds_diff'] = df[cols_interest[11:22]].apply(lambda x: x[x==1].sum()/11., axis=1)
df['mat_hds_diff2'] = df[cols_interest[:11]].apply(lambda x: x[x==1].sum()/11., axis=1)
df['regression'] = reg.predict(df[['mat_hds','mat_hds_diff','mat_hds_diff2']])
df = df.drop(df.index[(df.cum_sum < 0)])
df = df.drop(liveIn.index[liveIn.m5a2 == 1])

In [ ]:
df

In [ ]:
df.n5g1b.sum()
df[df.n5g1b > 0]

In [ ]:
print df_final.m5a2.loc[df[df.n5g1b > 0].index].value_counts()
print df_final.m5a2.value_counts()

In [ ]:
print df_final.f5a2.loc[df[df.n5g1b > 0].index].value_counts()
print df_final.f5a2.value_counts()

In [ ]:
print df_final.f5a2.loc[df[(df.m5f23a == 0)].index].value_counts()
print df_final.f5a2.loc[df[(df.m5f23a == 0) & (df.n5g1b == 0)].index].value_counts()
print df_final.f5a2.value_counts()

In [ ]:
matHard = df_test.materialHardship.loc[df.index].dropna()
matHard.shape

In [ ]:
train = df.loc[matHard.index]
train.shape

In [ ]:
#X = train.apply(lambda x: x.cat.codes)
#X['sum'] = X.sum(axis=1)
X = pd.DataFrame(train)
Y = matHard
keys = dict(zip(Y.unique(),range(len(Y.unique()))))
keys_back = dict(zip(range(len(Y.unique())), Y.unique()))
Y = Y.apply(lambda x: keys[x])

In [ ]:
def checkNegative(x):
    return isinstance(x, basestring) and x.startswith('-')

In [ ]:
chi2, pval = feature_selection.chi2(X,Y)
feat_indx = X.columns[~np.isnan(chi2)]
chi2_nona = chi2[~np.isnan(chi2)]
pval_nona = pval[~np.isnan(chi2)]
n_unique = X.apply(lambda x: len(x.unique()))
n_nan = X.apply(lambda x: x.isnull().sum())
#n_other = train.applymap(checkNegative).sum()

feat_rank = pd.DataFrame({
    'chi2': chi2_nona, 
    'pval': pval_nona,
    'unqe': n_unique,
    'n_nan': n_nan,
#    'other': n_other,
}, index = feat_indx)

In [ ]:
ordered = feat_rank.sort_values('pval')
print ordered

In [ ]:
#mask = ordered.index.isin(['cum_sum'])
#cols = ordered.index[~mask]
cols = ordered.index[:10]
cutoff = 500
#clf = MultinomialNB()
clf = LinearSVC()
#clf = SVC(kernel = 'sigmoid', tol=.000001, verbose = True)
#clf = tree.DecisionTreeClassifier()
clf.fit(X[cols].iloc[:cutoff], Y[:cutoff])

In [ ]:
out = clf.predict(X[cols].iloc[cutoff:])
plt.hist(out)
Y[cutoff:].hist(alpha = 0.5)
print metrics.classification_report(Y[cutoff:], out)
print metrics.confusion_matrix(Y[cutoff:], out)

#print metrics.brier_score_loss(Y[cutoff:], out)
print metrics.mean_squared_error(Y[cutoff:].apply(lambda x: keys_back[x]), map(lambda x: keys_back[x], out) )

In [ ]:
keys

### Test for Who's Who in The PCG slot

In [ ]:
whoWith = pd.DataFrame(df_test.materialHardship)

In [ ]:
whoWith['m5a2'] = df_final.m5a2
whoWith['f5a2'] = df_final.f5a2
whoWith['n5a5'] = df_final.n5a5

In [ ]:
print whoWith.n5a5.value_counts()

In [ ]:
print whoWith[whoWith.materialHardship.apply(lambda x: str(x).startswith('0.54'))].n5a5.value_counts()
print whoWith[whoWith.materialHardship.apply(lambda x: str(x).startswith('0.45'))].n5a5.value_counts()
print whoWith[whoWith.materialHardship.apply(lambda x: str(x).startswith('0.36'))].n5a5.value_counts()
print whoWith[whoWith.materialHardship.apply(lambda x: str(x).startswith('0.27'))].n5a5.value_counts()
print whoWith[whoWith.materialHardship.apply(lambda x: str(x).startswith('0.18'))].n5a5.value_counts()
print whoWith[whoWith.materialHardship.apply(lambda x: str(x).startswith('0.09'))].n5a5.value_counts()
print whoWith[whoWith.materialHardship.apply(lambda x: str(x) == '0.0')].n5a5.value_counts()

In [ ]:
whoWith[whoWith.n5a5.apply(lambda x: str(x).startswith('1'))].materialHardship.value_counts()

In [ ]:
whoWith[['m5a2','f5a2', 'n5a5']] = whoWith[['m5a2','f5a2', 'n5a5']].applymap(lambda x: abs(int(x.split(' ')[0])))

In [ ]:
whoWith = whoWith.dropna()
whoWith['code'] = whoWith[['m5a2','f5a2','n5a5']].apply(lambda x: ''.join(x.astype(int).astype(str)), axis=1).astype(int)

In [ ]:
plt.plot(whoWith.code,whoWith.materialHardship,'.',markersize = 19)

In [ ]:
whoDist = whoWith.groupby(['code', 'materialHardship']).code.count()
z = [(x[0],x[1],y) for x,y in whoDist.iteritems()]

In [ ]:
xd,yd,zd = zip(*z)

In [ ]:
plt.scatter(xd,yd,zd)
plt.xlim([100,200])

In [ ]:
scat = pd.concat([train.mat_hds, matHard], axis = 1)
nums = scat.groupby(['mat_hds', 'materialHardship']).mat_hds.count()
z = [(x[0],x[1],y) for x,y in nums.iteritems()]
xd,yd,zd = zip(*z)
plt.scatter(xd,yd,zd)
plt.xlabel('initial')
plt.ylabel('final')

In [ ]:
nums

In [ ]:
df_comp = pd.DataFrame(df[['mat_hds', 'mat_hds_diff', 'mat_hds_diff2']])
df_comp['true'] = df_test.materialHardship

In [ ]:
df_comp = df_comp.dropna()

In [ ]:
b = 500
#reg = BayesianRidge(tol = 0.00000001)
reg = LassoCV()
#reg = LinearRegression()
X = df_comp[[
    'mat_hds', 
    'mat_hds_diff',
    'mat_hds_diff2'
]]
Y = df_comp['true']
reg.fit(X.iloc[b:], Y.iloc[b:])

In [ ]:
out_reg = reg.predict(X.iloc[:b])

In [ ]:
metrics.mean_squared_error(Y[:b], out_reg)

In [ ]:
plt.plot(out_reg,Y[b:],'.')

Notes: to-do
» need to combine with other primary-care-giver and father information.
» need to feed in the results from the linear regression (which we know to be incorrect) and try to correct them with the classifier.